In [ ]:
pip install httpx

In [ ]:
import sys
# Example: if your files are in Files/utils/ within the default Lakehouse
sys.path.append("/lakehouse/default/Files/utils/")
print(f"Added {sys.path[-1]} to Python path.")


In [36]:
import pandas as pd
import json
import logging
import asyncio
import httpx

from powerbi_api_utils import load_config, get_api_constants, get_access_token, get_paginated_data_async
from fabric_utils import save_to_fabric_warehouse

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def _clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardizes DataFrame column names."""
    if not df.empty:
        df.columns = [str(col).replace('.', '_').replace(' ', '_') for col in df.columns]
    return df

def _normalize_and_merge_json_column(df: pd.DataFrame, col_name: str, prefix: str) -> pd.DataFrame:
    """Expands a column containing JSON/dicts into new columns and merges back."""
    if df.empty or col_name not in df.columns:
        return df
    df[col_name] = df[col_name].apply(lambda x: x if isinstance(x, dict) and x else None)
    to_normalize = df.dropna(subset=[col_name]).copy()
    if to_normalize.empty:
        return df.drop(columns=[col_name], errors='ignore')
    normalized_df = pd.json_normalize(to_normalize[col_name]).add_prefix(prefix)
    df = df.drop(columns=[col_name])
    df = df.merge(normalized_df, left_index=True, right_index=True, how='left')
    return _clean_columns(df)

async def get_all_metadata_async(token: str, base_url: str, admin_base_url: str, verify_ssl: bool, include_report_app_users: bool = True):
    """Orchestrates the fetching of all Power BI metadata concurrently."""
    headers = {'Authorization': f'Bearer {token}'}
    
    async with httpx.AsyncClient(verify=verify_ssl) as client:
        # --- 1. Fetch Capacities ---
        logging.info("--- Fetching Tenant-Level Admin Data ---")
        capacities_data = await get_paginated_data_async(client, f"{admin_base_url}/capacities", headers)
        capacities_df = _clean_columns(pd.DataFrame(capacities_data))
        if 'users' in capacities_df.columns:
            capacities_df = capacities_df.drop(columns=['users'])
        if not capacities_df.empty and 'admins' in capacities_df.columns:
            capacities_df['admins'] = capacities_df['admins'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

        # --- 2. Fetch Gateways ---
        logging.info("--- Fetching Gateways and their details ---")
        gateways_data = await get_paginated_data_async(client, f"{base_url}/gateways", headers)
        
        gateway_tasks = [get_paginated_data_async(client, f"{base_url}/gateways/{gw.get('id')}/datasources", headers) for gw in gateways_data if gw.get('id')]
        all_datasources_results = await asyncio.gather(*gateway_tasks)
        
        all_gateway_datasources = []
        for gateway, datasources in zip(gateways_data, all_datasources_results):
            for ds in datasources:
                ds['gatewayId'] = gateway.get('id')
                all_gateway_datasources.append(ds)
        
        gateways_df = _clean_columns(pd.DataFrame(gateways_data)).drop(columns=['publicKey', 'gatewayAnnotation'], errors='ignore')
        
        # --- FIX: Correctly process gateway_datasources_df ---
        gateway_datasources_df = _clean_columns(pd.DataFrame(all_gateway_datasources))
        # First, flatten 'credentialDetails' as per the original logic
        gateway_datasources_df = _normalize_and_merge_json_column(gateway_datasources_df, 'credentialDetails', 'credentialDetails_')
        # Then, flatten 'details' if it exists
        if 'details' in gateway_datasources_df.columns:
            gateway_datasources_df = _normalize_and_merge_json_column(gateway_datasources_df, 'details', 'details_')

        # --- Fetch Gateway Datasource Users ---
        datasource_user_tasks = [
            (get_paginated_data_async(client, f"{base_url}/gateways/{ds.get('gatewayId')}/datasources/{ds.get('id')}/users", headers), ds.get('gatewayId'), ds.get('id'))
            for ds in all_gateway_datasources if ds.get('id')
        ]
        user_task_futures = [t[0] for t in datasource_user_tasks]
        all_datasource_users_results = await asyncio.gather(*user_task_futures)
        all_datasource_users = []
        for (task, gateway_id, ds_id), users in zip(datasource_user_tasks, all_datasource_users_results):
            for user in users:
                user.update({'gatewayId': gateway_id, 'datasourceId': ds_id})
                all_datasource_users.append(user)
        gateway_datasource_users_df = _clean_columns(pd.DataFrame(all_datasource_users))
        if 'users' in gateway_datasource_users_df.columns:
            gateway_datasource_users_df = gateway_datasource_users_df.drop(columns=['users'])
            
        # --- 3. Fetch Workspaces and related assets ---
        logging.info("--- Fetching Workspace-Level Data ---")
        workspaces_url = f"{admin_base_url}/groups"; params = {"$expand": "users,reports,datasets,dataflows", "$filter": "type eq 'Workspace' and state eq 'Active'", "$top": 5000}
        workspaces_data = await get_paginated_data_async(client, workspaces_url, headers, params=params)
        
        all_workspaces, all_workspace_users, all_reports_raw, all_datasets_raw, all_dataflows_raw = [], [], [], [], []
        for ws in workspaces_data:
            ws_id, ws_name = ws.get('id'), ws.get('name')
            all_workspaces.append({k: v for k, v in ws.items() if k in ['id', 'name', 'isOnDedicatedCapacity', 'capacityId', 'type', 'state']})
            for item_list, item_type in [(ws.get('users', []), all_workspace_users), (ws.get('reports', []), all_reports_raw), (ws.get('datasets', []), all_datasets_raw), (ws.get('dataflows', []), all_dataflows_raw)]:
                for item in item_list: item.update({'workspace_id': ws_id, 'workspace_name': ws_name}); item_type.append(item)

        # --- ENFORCE SCHEMAS: Create and clean DataFrames ---
        workspaces_df = _clean_columns(pd.DataFrame(all_workspaces))
        workspace_users_df = _normalize_and_merge_json_column(_clean_columns(pd.DataFrame(all_workspace_users)), 'profile', 'profile_')
        reports_df = _clean_columns(pd.DataFrame(all_reports_raw)); reports_df = reports_df.drop(columns=['users', 'subscriptions'], errors='ignore')
        datasets_df = _clean_columns(pd.DataFrame(all_datasets_raw)); datasets_df = datasets_df.drop(columns=['users', 'upstreamDatasets'], errors='ignore')
        dataflows_df = _clean_columns(pd.DataFrame(all_dataflows_raw)); dataflows_df = dataflows_df.drop(columns=['users'], errors='ignore')

        # --- 4. Fetch App and Report Users (if requested) ---
        report_users_df, apps_df, app_users_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        # (This section remains unchanged)
        if include_report_app_users:
            logging.info("--- Fetching App/Report Users Concurrently ---")
            apps_data = await get_paginated_data_async(client, f"{admin_base_url}/apps", headers, params={'$top': 5000})
            apps_df = _clean_columns(pd.DataFrame(apps_data))
            app_user_tasks = [(get_paginated_data_async(client, f"{admin_base_url}/apps/{app['id']}/users", headers), app['id'], app['name']) for index, app in apps_df.iterrows()] if not apps_df.empty else []
            report_user_tasks = [(get_paginated_data_async(client, f"{admin_base_url}/groups/{report['workspace_id']}/reports/{report['id']}/users", headers), report['workspace_id'], report['id'], report['name']) for index, report in reports_df.iterrows()] if not reports_df.empty else []
            all_user_tasks = [t[0] for t in app_user_tasks] + [t[0] for t in report_user_tasks]
            all_user_results = await asyncio.gather(*all_user_tasks)
            num_app_tasks = len(app_user_tasks)
            all_app_users = [{**user, 'app_id': app_id, 'app_name': app_name} for (task, app_id, app_name), users in zip(app_user_tasks, all_user_results[:num_app_tasks]) for user in users]
            app_users_df = _normalize_and_merge_json_column(_clean_columns(pd.DataFrame(all_app_users)), 'profile', 'profile_')
            all_report_users = [{**user, 'workspace_id': ws_id, 'report_id': report_id, 'report_name': report_name} for (task, ws_id, report_id, report_name), users in zip(report_user_tasks, all_user_results[num_app_tasks:]) for user in users]
            report_users_df = _normalize_and_merge_json_column(_clean_columns(pd.DataFrame(all_report_users)), 'profile', 'profile_')

        # --- FINAL STEP: Add extraction timestamp to all dataframes ---
        all_dfs = {"capacities": capacities_df, "gateways": gateways_df, "gateway_datasources": gateway_datasources_df, "gateway_datasource_users": gateway_datasource_users_df, "workspaces": workspaces_df, "workspaces_users": workspace_users_df, "reports": reports_df, "report_users": report_users_df, "datasets": datasets_df, "dataflows": dataflows_df, "apps": apps_df, "app_users": app_users_df}

        # Convert timestamp to an integer (Unix epoch seconds) to match the existing table schema
        extraction_ts_int = int(pd.Timestamp.now().timestamp())

        for name, df in all_dfs.items():
            if not df.empty:
                df['extraction_timestamp'] = extraction_ts_int # Assign the integer

        return all_dfs

async def run_extraction_to_fabric(include_report_app_users: bool = True):
    """Main async function to run the metadata extraction and save to Fabric Warehouse."""
    try:
        logging.info("1. Getting SparkSession..."); spark_session = globals()['spark'];
        logging.info("2. Loading configuration..."); config = load_config(); AUTHORITY, SCOPE, BASE_URL, ADMIN_BASE_URL = get_api_constants(config["TENANT_ID"]);
        logging.info("3. Authenticating Power BI..."); access_token = get_access_token(config["CLIENT_ID"], config["CLIENT_SECRET"], AUTHORITY, SCOPE);
        logging.info(f"4. User/App data extraction set to: {'Include' if include_report_app_users else 'Exclude'}")
        logging.info("5. Starting concurrent metadata extraction...")
        all_data_dfs = await get_all_metadata_async(access_token, BASE_URL, ADMIN_BASE_URL, config["VERIFY_SSL"], include_report_app_users=include_report_app_users)
        logging.info("\n6. Saving extracted metadata to Microsoft Fabric Warehouse...")
        save_to_fabric_warehouse(all_data_dfs, warehouse_schema="powerbi_metadata", spark=spark_session)
        logging.info("✅ 7. Metadata extraction and saving to Fabric Warehouse completed.")
    except Exception as e:
        logging.error(f"\n❌ An error occurred during the process: {e}", exc_info=True)
        raise

StatementMeta(, dffadbfa-4411-41b3-b4bc-74a6250bf888, 38, Finished, Available, Finished)

###### **Change include_report_app_users from False to True if you want to include them.**

In [ ]:
# run_extraction_to_fabric(include_report_app_users=False)
await run_extraction_to_fabric(include_report_app_users=False)